<a href="https://colab.research.google.com/github/pompaFunebris/ContextBasedCompression/blob/main/Lempel_Ziv_Welch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Lempel Ziv Welch algorithm**

> LZ77 is a loseless data compression algorithm created by Abraham Lempel, Jacob Ziv and Terry Welch. It was published by Welch in 1984 as an improved implementation of the LZ78 algorithm published by Lempel and Ziv in 1978.  is the algorithm of the Unix file compression utility compress and is used in the GIF image format.

The LZW algorithm works by building a dictionary of frequently occurring patterns in the input data. It starts with an initial dictionary containing single-character patterns and then grtadually expands it to onclude longer patterns encountered.

During compression, the LZW algorithm scans the input data and identifies the longest patterns that have not been seen before. It replaces these patterns with shorter codes that reference entries in the dictionary. By using these codes, the algorithm achieves compression by representing repetitive patterns with shorter codes instead of the original longer sequences. The dictionary is never sent between the two sides, but is constructed independently by the sender and the receiver.

*Sources*:
* https://marknelson.us/posts/2011/11/08/lzw-revisited.html
* https://en.wikipedia.org/wiki/Lempel%E2%80%93Ziv%E2%80%93Welch
* https://www.mbit.edu.in/wp-content/uploads/2020/05/data_compression.pdf
* https://www.youtube.com/watch?v=j2HSd3HCpDs

In [ ]:

def lzw_encoder(input_sequence, initialize_dict_size, max_dict_size=32767):
  """
  Encode an input sequence using the LZW compression algorithm.

  Args:
      input_sequence (str): The input sequence to be encoded.
      initialize_dict_size (int): The initial dictionary size.
      max_dict_size (int, optional): The maximum dictionary size. Defaults to 32767.

  Returns:
      tuple: A tuple containing the encoded sequence and the encoder dictionary.
  """

  # Initialize the encoder dictionary with 7 bit ASCII alphabet
  encoder_dictionary = {}
  for i in range(initialize_dict_size):
    char = chr(i)
    encoder_dictionary[char] = i

  encoded_sequence = []
  current_sequence = ""

  # We will be adding new codes to the existing dictionary.
  next_code = initialize_dict_size + 1

  for character in input_sequence:
    current_sequence = current_sequence + character

    # We found some matching string, but after adding the new character it is no longer
    # found in the dictionary, so we encode the longest maching string and add the
    # new codeword with one more letter to the dictionary.
    if current_sequence not in encoder_dictionary:

      # Create new codeword that is one character longer than the matched sequence
      # if there is still space in the dictionary.
      if next_code <= max_dict_size:
        encoder_dictionary[current_sequence] = next_code
        next_code = next_code + 1

      # We encode the longest matching string that was found in the dictionary.
      matched_sequence = current_sequence[:-1]
      current_sequence = character
      encoded_sequence.append(encoder_dictionary[matched_sequence])

  # Add the code for the final sequence to the dictionary.
  encoded_sequence.append(encoder_dictionary[current_sequence])

  return encoded_sequence, encoder_dictionary

In [ ]:
def lzw_decoder(encoded_sequence, initialize_dict_size, max_dict_size=32767):
  """
  Decode an encoded sequence using the LZW compression algorithm.

  Args:
      encoded_sequence (list): The encoded sequence to be decoded.
      initialize_dict_size (int): The initial dictionary size.
      max_dict_size (int, optional): The maximum dictionary size. Defaults to 32767.

  Returns:
      tuple: A tuple containing the decoded string and the decoder dictionary.
  """
  decoder_dictionary = {}
  for i in range(initialize_dict_size):
    char = chr(i)
    decoder_dictionary[i] = char

  next_code = initialize_dict_size + 1
  previous_string = ""
  decoded_string = ""

  for code_word in encoded_sequence:
    if code_word not in decoder_dictionary:
      decoder_dictionary[code_word] = previous_string + previous_string[0]

    decoded_string += decoder_dictionary[code_word]

    if previous_string and next_code <= max_dict_size:
      decoder_dictionary[next_code] = previous_string + decoder_dictionary[code_word][0]
      next_code += 1

    previous_string = decoder_dictionary[code_word]

  return decoded_string, decoder_dictionary



In [ ]:
encoded_sequence, encoder_dict = lzw_encoder("karawana karwasz-kara", 128)
print(encoded_sequence)

decoded_sequence, decoder_dict = lzw_decoder(encoded_sequence, 128)
print(decoded_sequence)

[107, 97, 114, 97, 119, 97, 110, 97, 32, 129, 114, 133, 115, 122, 45, 138, 97]
karawana karwasz-kara


In [ ]:
import requests
import zipfile

url = 'http://corpus.canterbury.ac.nz/resources/cantrbry.zip'
r = requests.get(url, allow_redirects=True)

open('cantrbry.zip', 'wb').write(r.content)
with zipfile.ZipFile('cantrbry.zip', 'r') as zip_ref:
    zip_ref.extractall('cantrbry')

with open('cantrbry/alice29.txt', encoding='utf8') as f:
    text = f.read()
f.close()

text = text.upper()
print(text[:1000])





                ALICE'S ADVENTURES IN WONDERLAND

                          LEWIS CARROLL

               THE MILLENNIUM FULCRUM EDITION 2.9




                            CHAPTER I

                      DOWN THE RABBIT-HOLE


  ALICE WAS BEGINNING TO GET VERY TIRED OF SITTING BY HER SISTER
ON THE BANK, AND OF HAVING NOTHING TO DO:  ONCE OR TWICE SHE HAD
PEEPED INTO THE BOOK HER SISTER WAS READING, BUT IT HAD NO
PICTURES OR CONVERSATIONS IN IT, `AND WHAT IS THE USE OF A BOOK,'
THOUGHT ALICE `WITHOUT PICTURES OR CONVERSATION?'

  SO SHE WAS CONSIDERING IN HER OWN MIND (AS WELL AS SHE COULD,
FOR THE HOT DAY MADE HER FEEL VERY SLEEPY AND STUPID), WHETHER
THE PLEASURE OF MAKING A DAISY-CHAIN WOULD BE WORTH THE TROUBLE
OF GETTING UP AND PICKING THE DAISIES, WHEN SUDDENLY A WHITE
RABBIT WITH PINK EYES RAN CLOSE BY HER.

  THERE WAS NOTHING SO VERY REMARKABLE IN THAT; NOR DID ALICE
THINK IT SO VERY MUCH OUT OF THE WAY TO HEAR THE RABBIT SAY TO
ITSELF, `OH DEAR!  OH DEAR!  I SHALL BE LAT

In [ ]:
encoded_sequence, encoder_dict = lzw_encoder(text, 128)
print(encoded_sequence[:50])

decoded_sequence, decoder_dict = lzw_decoder(encoded_sequence, 128)
print(decoded_sequence[:1000]+"...")

[10, 129, 10, 32, 132, 133, 134, 135, 32, 65, 76, 73, 67, 69, 39, 83, 137, 68, 86, 69, 78, 84, 85, 82, 69, 144, 73, 78, 32, 87, 79, 78, 68, 69, 82, 76, 65, 160, 129, 136, 168, 169, 135, 76, 69, 87, 73, 144, 67, 65]




                ALICE'S ADVENTURES IN WONDERLAND

                          LEWIS CARROLL

               THE MILLENNIUM FULCRUM EDITION 2.9




                            CHAPTER I

                      DOWN THE RABBIT-HOLE


  ALICE WAS BEGINNING TO GET VERY TIRED OF SITTING BY HER SISTER
ON THE BANK, AND OF HAVING NOTHING TO DO:  ONCE OR TWICE SHE HAD
PEEPED INTO THE BOOK HER SISTER WAS READING, BUT IT HAD NO
PICTURES OR CONVERSATIONS IN IT, `AND WHAT IS THE USE OF A BOOK,'
THOUGHT ALICE `WITHOUT PICTURES OR CONVERSATION?'

  SO SHE WAS CONSIDERING IN HER OWN MIND (AS WELL AS SHE COULD,
FOR THE HOT DAY MADE HER FEEL VERY SLEEPY AND STUPID), WHETHER
THE PLEASURE OF MAKING A DAISY-CHAIN WOULD BE WORTH THE TROUBLE
OF GETTING UP AND PICKING THE DAISIES, WHEN SUDDENLY A 

In [ ]:
def analyze_encoder_dictionary(encoder_dict, num_samples=30):
    codeword_count = len(encoder_dict)
    total_length = sum(len(codeword) for codeword in encoder_dict)
    average_length = total_length / codeword_count

    sample_sequences = []
    sample_codes = []

    # Get a random sample of sequences and their codes
    import random
    keys = list(encoder_dict.keys())
    random.shuffle(keys)
    sample_keys = keys[:num_samples]

    for key in sample_keys:
        sample_sequences.append(key)
        sample_codes.append(encoder_dict[key])

    return codeword_count, average_length, sample_sequences, sample_codes

# Call the function to analyze the encoder_dict
codeword_count, average_length, sample_sequences, sample_codes = analyze_encoder_dictionary(encoder_dict)

print("Number of codewords in encoder_dict:", codeword_count)
print("Average length of codeword:", average_length)
print("Sample sequences and their codes:")
for sequence, code in zip(sample_sequences, sample_codes):
    print(f"Sequence: {sequence} | Code: {code}")
print("Efficiency: " + str(7*average_length/15))


Number of codewords in encoder_dict: 32767
Average length of codeword: 5.356639301736503
Sample sequences and their codes:
Sequence: L JU | Code: 10447
Sequence: OLDIER | Code: 23189
Sequence:  YOUR  | Code: 8705
Sequence:  ONE, | Code: 32181
Sequence: 
FAC | Code: 4624
Sequence: OR A
 | Code: 19248
Sequence: 
DORM | Code: 20694
Sequence: E FOR S | Code: 14279
Sequence: 
    AN | Code: 28809
Sequence: BATS | Code: 2105
Sequence: 
VE | Code: 12963
Sequence:  NEI | Code: 21473
Sequence: A,'  | Code: 31229
Sequence: REPEA | Code: 15111
Sequence: SHE WAS NO | Code: 6326
Sequence: OWN
 | Code: 29740
Sequence: `FI | Code: 30269
Sequence: ORI | Code: 28656
Sequence:  MAT | Code: 24217
Sequence: BUT SHE CO | Code: 26341
Sequence: ANT A | Code: 21083
Sequence: MENT  | Code: 9914
Sequence:                | Code: 4262
Sequence: ALICE'S  | Code: 24851
Sequence: , " | Code: 9068
Sequence: ER ` | Code: 9398
Sequence: RE THE J | Code: 29814
Sequence: T I H | Code: 18597
Sequence: , ON B | Code: 21358